In [1]:
import sys
import re
from tqdm import tqdm
import random
import json

import time
from tqdm import tqdm
import requests
import faiss
import numpy as np
from simcse import SimCSE
from shared.prompt import instance
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM
from transformers import pipeline
from KNN_RE import get_train_example, find_knn_example, find_lmknn_example
from load_model import get_emb,Args
from label_explain import SemEval_label2exp
sys.path.append("/group/40064/johnbli/Code/UGC")
from examples.gpt_http import retry_request_openai_summary

params = {
    # 任务信息
    "env": "production",  # 必填，（取值范围：production、development）
    "task_name": "逃出精神病院",
    "task_type": "文本生成任务",  # 必填，任务类型（取值范围：qa任务、文本生成任务、整段对话生产、多轮对话、其他）
    "task_description": "",
    "business_id":"134"
}
import copy
import math

/data/miniconda3/envs/env_clone-3.9.16/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/data/miniconda3/envs/env_clone-3.9.16/lib/python3.9/site-packages/torch_npu/utils/path_manager.py:82: UserWarning: Warning: The /usr/local/Ascend/ascend-toolkit/latest owner does not match the current user.
  warnings.warn(f"Warning: The {path} owner does not match the current user.")
/data/miniconda3/envs/env_clone-3.9.16/lib/python3.9/site-packages/torch_npu/utils/path_manager.py:82: UserWarning: Warning: The /usr/local/Ascend/ascend-toolkit/8.0.RC2.alpha003/x86_64-linux/ascend_toolkit_install.info owner does not match the current user.
  warnings.warn(f"Warning: The {path} owner does not match the current user.")


In [2]:
dataset_path = "/group/40064/johnbli/Code/GPT-RE/dataset/semeval_gpt/test.json"

test_data = []
with open(dataset_path, "r") as f:
    for line in f:
        line = json.loads(line)
        test_data.append(line)


In [3]:
test_data[0]

{'sentences': [['the',
   'most',
   'common',
   'audits',
   'were',
   'about',
   'waste',
   'and',
   'recycling',
   '.']],
 'ner': [[[3, 3, 'sub'], [6, 6, 'obj']]],
 'predicted_ner': [[[3, 3, 'sub'], [6, 6, 'obj']]],
 'relations': [[[3, 3, 6, 6, 'Message-Topic']]],
 'doc_key': '0oftrain'}

In [4]:
semeval_idtoprompt = {0:"NONE",1:"CAUSE AND EFFECT", 2:"COMPONENT AND WHOLE", 3:"ENTITY AND DESTINATION",4:"ENTITY AND ORIGIN",5:"PRODUCT AND PRODUCER",6:"MEMBER AND COLLECTION",7:"MESSAGE AND TOPIC",8:"CONTENT AND CONTAINER",9:"INSTRUMENT AND AGENCY"}
semeval_reltoid = {"Other":0,"Cause-Effect":1, "Component-Whole":2, "Entity-Destination":3, "Entity-Origin":4, "Product-Producer": 5, "Member-Collection":6, "Message-Topic": 7, "Content-Container":8, "Instrument-Agency":9}


In [5]:
GPT_PROMPT = '''
You are a relation extraction expert specializing in the field of relation extraction, responsible for identifying and extracting the relationship between entities from text to support applications such as information retrieval, knowledge graph construction and data analysis. The following is the entity information:

Entity 1: {ent1}

Entity 2: {ent2}

The categories of relations include the following: {rels}

Your task is to process the text given below to accurately identify and extract the relationship between entity 1 and entity 2 in the text from the list of relation categories.

**Requirements**
You must accurately identify the type of relationship between entities in the sentence.
The response should not contain any information other than the entity category.

**Examples**
{examples}

Please judge the relationship between the following sentences based on the above examples:
**Input**
{sen}

Output format:
Relation: XXX

'''

In [6]:
LLaMA_PROMPT = '''
You are a system prompt generation expert specializing in the field of relation extraction, responsible for identifying and extracting the relationship between entities from text to support applications such as information retrieval, knowledge graph construction and data analysis. The following is the entity information:

Entity 1: {ent1}

Entity 2: {ent2}

The categories of relations include the following: {rels}

Your task is to process the text given below to accurately identify and extract the relationship between entity 1 and entity 2 in the text from the list of relation categories.

**Examples**
{examples}

**Requirements**
You must accurately identify the type of relationship between entities in the sentence
The response should not contain any information other than the entity category

Please judge the relationship between the following sentences based on the above examples:
**Input**
{sen}

Output format:
Relation: XXX

'''

**GPT**

In [7]:
def gen_first_dia(ent1, ent2, rels, examples, sen):
    prompt = GPT_PROMPT.format(ent1 = ent1,ent2 = ent2,rels=rels,examples=examples, sen = sentence)
    # print(prompt)
    res = retry_request_openai_summary(prompt, **params)[0]
    res_lst = [item for item in res.split("\n") if item]
    return res_lst

def gen_GPT_first_chat(ent1, ent2, rels,examples, sen):
    res_lst = gen_first_dia(ent1, ent2, rels,examples, sen)
    return res_lst

**Qwen2.5-70B**

In [8]:
def predict_chat(messages, model_namespace='Production', model_service_name='DataGPTv1ServerVllm'):

    req = {
        "messages": messages,
        "model_version": "default",
        "task_id": time.time(),
    }
    # print(req)

    # 拼接请求url
    url = f'http://11.135.133.241:13023/v1/chat'

    
    # 发起请求
    ret = requests.request("POST", url, headers={"Content-Type": "application/json"},
                           data=json.dumps(req))
    # print(type(ret))
    # print(ret.status_code)

    result = json.loads(ret.text)

    return result["choices"][0]["message"]["content"]


def predict_chat_prompt(prompt, model_namespace='Production', model_service_name='DataGPTv1ServerVllm'):

    req = {
        "prompt": prompt,
        "model_version": "default",
        "task_id": time.time(),
    }
    # print(req)

    # 拼接请求url
    url = f'http://11.135.133.241:13023/v1/chat'

    
    # 发起请求
    ret = requests.request("POST", url, headers={"Content-Type": "application/json"},
                           data=json.dumps(req))
    # print(type(ret))
    # print(ret.status_code)

    result = json.loads(ret.text)

    return result["choices"][0]["message"]["content"]

In [9]:
def get_Qwen_res(ent1, ent2, rels, sen):
    # print(chai)
    prompt = GPT_PROMPT.format(ent1 = ent1,ent2 = ent2,rels=rels, sen = sen)
    messages = [{'role': 'system', 'content': 'You are an AI assistant '}, {'role': 'user', 'content': prompt}]
    res_lst = predict_chat(messages)
    return [res_lst]

**Qwen2.5-7B**

In [10]:
model_name_or_path = '/group/40064/heyfonli/model_hub/Qwen2.5-7B-Instruct/'
# model_name_or_path = "/group/40064/johnbli/LLM-based-models/LLama3/Meta-Llama-3-8B-Instruct"

In [11]:
model = AutoModelForCausalLM.from_pretrained(model_name_or_path, device_map='npu:1', trust_remote_code=True)

Loading checkpoint shards: 100%|██████████| 4/4 [00:33<00:00,  8.49s/it]


In [12]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [13]:
def get_Qwen_res_7b(ent1, ent2, rels, examples, sentence):
    prompt = GPT_PROMPT.format(ent1 = ent1,ent2 = ent2,rels=rels,examples=examples, sen = sentence)
    messages = [{'role': 'system', 'content': 'You are an AI assistant '}, {'role': 'user', 'content': prompt}]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    # print(text)
    model_inputs = tokenizer([text], return_tensors="pt").to('npu:1')
    generated_ids = model.generate(
            **model_inputs,
            max_new_tokens=512,
            do_sample=True,
            temperature=0.9,
            top_p = 0,
            top_k = 1
        )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return [response]


In [14]:
TRAIN_SUP = '''
Sample {num}:
    Entity 1: {ent1}
    Entity 2: {ent2}
    Input:{sen}
    Output:Relation: {label}
'''

In [15]:
use_instance = True
use_knn = True
use_pipline = False
k_sample = 5

example_dict = get_train_example("/group/40064/johnbli/Code/GPT-RE/dataset/semeval_gpt/train.json", semeval_reltoid, 0)
train_list = [x for y in example_dict.values() for x in y]
# train_list = [x for x in train_list if semeval_reltoid[x["relations"][0][0][4]] != 0]
train_sentences = [instance(x).sentence for x in train_list]
train_reference = [instance(x).reference for x in train_list]
train_dict = {instance(x).reference:x for x in train_list}


In [16]:
if use_knn:
    knn_model = SimCSE("/group/40064/johnbli/bert-based-models/sup-simcse-roberta-large")
    knn_model.build_index(train_reference, device="cpu")

/data/miniconda3/envs/env_clone-3.9.16/lib/python3.9/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
01/04/2025 21:01:50 - INFO - simcse.tool -   Encoding embeddings for sentences...
100%|██████████| 102/102 [02:14<00:00,  1.32s/it]
01/04/2025 21:04:04 - INFO - simcse.tool -   Building index...
01/04/2025 21:04:04 - INFO - simcse.tool -   Use CPU-version faiss
01/04/2025 21:04:04 - INFO - simcse.tool -   Finished


In [17]:
import torch
if use_pipline:

    # res = faiss.StandardGpuResources()  # 不再需要初始化GPU资
    # gpu_index_flat = faiss.index_cpu_to_gpu(res, 0, index_flat)  # 不需要转移到GPU
    index_flat = faiss.IndexFlatL2(768)  # 创建一个使用L2距离的平面索引
    cpu_index_flat = index_flat  # 直接使用CPU索引
    args =Args()
    dataset = 'SemEval'
    retriever_plm = "/group/40064/johnbli/bert-based-models/bert_base_uncased"
    device = torch.device("npu:7")
    train_samples = get_emb(args, train_list, dataset, retriever_plm, k_sample, device) #使用全部数据集
    # train_samples = get_emb(args, "", dataset, retriever_plm, k_sample, device) #使用部分数据集

    embed_list = np.array(train_samples)
    cpu_index_flat.add(embed_list)  # 使用CPU索引添加特征向量


In [18]:
def get_example(item):
    sentences_list = item["sentences"][0]
    ent1 = " ".join(sentences_list[item["ner"][0][0][0]:item["ner"][0][0][1]+1])
    ent2 = " ".join(sentences_list[item["ner"][0][1][0]:item["ner"][0][1][1]+1])
    sentence = " ".join(sentences_list)
    label = item["relations"][0][0][-1]
    return ent1,ent2,sentence,label

In [19]:
if use_knn:
    knn_select = find_knn_example(knn_model, test_data, train_dict, k_sample, True)

100%|██████████| 43/43 [00:51<00:00,  1.19s/it]


In [20]:
if use_pipline:
    test_samples = get_emb(args, test_data, dataset, retriever_plm, k_sample, device)
    D, I = cpu_index_flat.search(test_samples, k_sample)
    lm_select = []
    for j in range(I.shape[0]):
        j_select = [train_dict[train_sentences[i]] for i in I[j,:k_sample]]
        lm_select.append(j_select)
    # lm_select = find_lmknn_example(cpu_index_flat, test_data, train_dict, train_sentences, k_sample)

In [21]:
Labels = []
predict = []
num = 0
ids = 0
for test_item in tqdm(test_data):

    sentences_list = test_item["sentences"][0]
    ent1 = " ".join(sentences_list[test_item["ner"][0][0][0]:test_item["ner"][0][0][1]+1])
    ent2 = " ".join(sentences_list[test_item["ner"][0][1][0]:test_item["ner"][0][1][1]+1])
    sentence = " ".join(sentences_list)
    rels = ""
    rels_lst = {}
    for k,v in semeval_idtoprompt.items():
        rels = rels + v + '; '
        rels_lst[v] = k
    # 更详细的关系描述
    # for k,v in SemEval_label2exp.items():  
    #     rels = rels + k + ":" + v + '\n'
    label = semeval_reltoid[test_item["relations"][0][0][-1]]
    Labels.append(label)
    # 添加instance
    example = ""
    if use_instance:
        if use_knn:
            selected_items = knn_select[ids]
            ids += 1
            # print(selected_items)
        elif use_pipline:
            selected_items = lm_select[ids]
            ids += 1
        else:
            selected_items = random.sample(train_list, k_sample)
        for i,item in enumerate(selected_items):
            train_ent1,train_ent2,train_sentence,train_label = get_example(item)
            train_label = semeval_idtoprompt[semeval_reltoid[train_label]]
            example = example + TRAIN_SUP.format(num = i, ent1 = train_ent1,ent2 = train_ent2,label=train_label, sen = train_sentence)

    # res_lst = get_Qwen_res(ent1, ent2, rels, sentence)    #Qwen2.5-70b
    # res_lst = gen_GPT_first_chat(ent1, ent2, rels, example, sentence)     #GPT-70b
    res_lst = get_Qwen_res_7b(ent1, ent2, rels,example, sentence)        #Qwen2.5-7b
    
    res = res_lst[0].replace("Relation: ","") # 无理由
    # res = res_lst[0].split('\n')[-1].replace("Relation: ","")  #有理由
    # print(res)

    if res not in rels_lst:
        num += 1
        predict.append(0 if label!=0 else 1)
    else:
        predict.append(rels_lst[res])


  0%|          | 0/2717 [00:00<?, ?it/s]/data/miniconda3/envs/env_clone-3.9.16/lib/python3.9/site-packages/transformers/generation/logits_process.py:464: UserWarning: AutoNonVariableTypeMode is deprecated and will be removed in 1.10 release. For kernel implementations please use AutoDispatchBelowADInplaceOrView instead, If you are looking for a user facing API to enable running your inference-only workload, please use c10::InferenceMode. Using AutoDispatchBelowADInplaceOrView in user code is under risk of producing silent wrong result in some edge cases. See Note [AutoDispatchBelowAutograd] for more details. (Triggered internally at torch_npu/csrc/aten/common/TensorFactories.cpp:74.)
  sorted_indices_to_remove[..., -self.min_tokens_to_keep :] = 0
[W VariableFallbackKernel.cpp:51] Warning: CAUTION: The operator 'aten::isin.Tensor_Tensor_out' is not currently supported on the NPU backend and will fall back to run on the CPU. This may have performance implications. (function npu_cpu_fallb

In [22]:
from sklearn.metrics import f1_score

# 计算 F1 值
macro_f1 = f1_score(Labels, predict, average='macro')  # 'weighted' 考虑各类标签的不平衡问题
micro_f1 = f1_score(Labels, predict, average='micro')  # 'weighted' 考虑各类标签的不平衡问题

print("micro_f1 Score:", micro_f1)
print("macro_f1 Score:", macro_f1)

micro_f1 Score: 0.6013986013986014
macro_f1 Score: 0.5975918708678143


In [23]:
res

'ENTITY AND DESTINATION'